<a href="https://colab.research.google.com/github/philipilono/qosf-negative-numbers/blob/main/QOSF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install qiskit ipywidgets
!pip install -U qiskit-aer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 66.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.7/112.7 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 58.4 MB/s eta 0:00:00


In [2]:
import qiskit as qk
from qiskit import QuantumCircuit, QuantumRegister, Aer, transpile
from qiskit.quantum_info.operators import Operator
from qiskit.providers.aer import AerSimulator
import numpy as np

In [3]:
def create_oracle(num_qubits: int, min_num: int):

  qc = QuantumCircuit(QuantumRegister(num_qubits))
  #n qubits can be in 2^n possible states
  #creates a matrix to accomodate all possible states
  matrix = np.eye(2 ** num_qubits)
  m=-1*(min_num)
  # flips the sign of the element at position [m][m], creating a phase inversion for the corresponding quantum state.
  matrix[m][m] = -matrix[m][m]
  qc.unitary(Operator(matrix), range(num_qubits))

  return qc.to_instruction()

def create_diffuser(num_qubits: int):
    #qr = QuantumRegister(num_qubits)
    qc = QuantumCircuit(QuantumRegister(num_qubits))

    #prepares an equal superposition of all possible states and then flips the states
    for qubit in range(num_qubits):
       qc.h(qubit)
       qc.x(qubit)

    qc.barrier(range(num_qubits))
    qc.h(num_qubits-1)
    qc.mct(list(range(num_qubits-1)), num_qubits-1)
    qc.h(num_qubits-1)
    qc.barrier(range(num_qubits))

    for qubit in range(num_qubits):
        qc.x(qubit)
        qc.h(qubit)

    return qc.to_instruction()

In [6]:
from typing import List
def find_negative_numbers(numbers: List[int]):
    min_number = min(numbers)

    # log base of 2 computes how many bits are required to represent the indices of the list
    # and the value is then rounded to the upper nearest integer to ensure there is enough qubits
    num_qubits = int(np.ceil(np.log2(len(numbers)))) + 1
    qc = QuantumCircuit(num_qubits)

    for i in range(num_qubits):
        qc.h(i)

    #Grover's search algorithm
    niterations = int(np.floor(np.pi/4 * np.sqrt(2**num_qubits)))
    for i in range(niterations):
        qc.append(create_oracle(num_qubits, min_number), range(num_qubits))
        qc.append(create_diffuser(num_qubits), range(num_qubits))

    qc.measure_all()
    aer_sim = AerSimulator()
    compiled_circuit = transpile(qc, aer_sim)
    result = aer_sim.run(compiled_circuit).result()
    counts = result.get_counts()

    target = -(int(max(counts, key=counts.get), 2))

    if min_number == target:
        return True
    else:
        return False

A = find_negative_numbers([1,-3,2,15])
print(A)

B = find_negative_numbers([1,4,8,11])
print(B)

C = find_negative_numbers([-15,-14,2,-1, 1,4,8,11,1,-3,2,15, -15,-14,2,-1, 1,4,8,11,1,-3,2,15])
print(C)

D = find_negative_numbers([2, 3, 1, 4,2, 7, 4, 5])
print(D)

E = find_negative_numbers([2, 3, 1,-4,2, 7, 4, 5])
print(E)

F = find_negative_numbers([3, 7, -2, 10, -5])
print(F)

G = find_negative_numbers([13, 7, -2, -10, -5])
print(G)

True
False
True
False
True
True
True
